<a href="https://colab.research.google.com/github/Guiils/analise_de_dados/blob/main/projeto_brasileir%C3%A3o_flamengo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparação de bibliotecas

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=7976f59dd6c432b497612875cdd0dd35e842861ec646dc4e77e7a9f778293219
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


In [ ]:

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import isnull
from pyspark.sql.types import TimestampType, DateType, FloatType, DoubleType, StructType, StructField, StringType, IntegerType
from google.cloud import storage
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

##Conexões com GCP e Spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/brasileirao-analises-64875f22fe87.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
path_jogos = '/content/drive/MyDrive/Projeto Braileirao/brutos/campeonato-brasileiro-full.csv'
path_estatisticas = '/content/drive/MyDrive/Projeto Braileirao/brutos/campeonato-brasileiro-estatisticas-full.csv'

In [ ]:
#configurar a variavel de ambiente (sessão) Pyspark
spark = (SparkSession.builder
                     .master('local')
                     .appName('analises_brasileirao')
                     .config('spark.ui.port', '4050')
                     .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                     .getOrCreate()
)

In [ ]:
spark

##Extraindo data frame da Cloud

In [ ]:
#Importando dataframe do GCP
df_jogos = (
         spark.read
         .format('csv')
         .option('delimiter', ',') 
         .option('header', 'true')
         .option('encoding', 'utf-8')
         .option('inferschema', 'true')
         .load(path_jogos)
)


In [ ]:
df_estatisticas = (
         spark.read
         .format('csv')
         .option('delimiter', ',') 
         .option('header', 'true')
         .option('encoding', 'utf-8')
         .option('inferschema', 'true')
         .load(path_estatisticas)
)

##Analise exploratória

Analise df_jogos

In [ ]:
df_jogos.show(truncate = False)

+---+------+---------+-------------------+-------------+-------------+-----------------+------------------+----------------+-----------------+-------------+------------------+---------------+----------------+---------------+----------------+
|ID |rodata|data     |hora               |mandante     |visitante    |formacao_mandante|formacao_visitante|tecnico_mandante|tecnico_visitante|vencedor     |arena             |mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|
+---+------+---------+-------------------+-------------+-------------+-----------------+------------------+----------------+-----------------+-------------+------------------+---------------+----------------+---------------+----------------+
|1  |1     |29/3/2003|2023-05-18 16:00:00|Guarani      |Vasco        |null             |null              |null            |null             |Guarani      | Brinco de Ouro   |4              |2               |SP             |RJ              |
|2  |1     |29/3/2003|2023-05-18

In [ ]:
df_jogos.select(F.col('mandante')).distinct().count()

45

In [ ]:
df_jogos.select(F.col('mandante')).distinct().orderBy(F.col('mandante')).show(100)

+---------------+
|       mandante|
+---------------+
|     America-MG|
|     America-RN|
|   Athletico-PR|
|    Atletico-GO|
|    Atletico-MG|
|           Avai|
|          Bahia|
|        Barueri|
|    Botafogo-RJ|
|     Bragantino|
|    Brasiliense|
|            CSA|
|          Ceara|
|    Chapecoense|
|    Corinthians|
|       Coritiba|
|       Criciuma|
|       Cruzeiro|
|         Cuiaba|
|    Figueirense|
|       Flamengo|
|     Fluminense|
|      Fortaleza|
|          Goias|
|         Gremio|
|Gremio Prudente|
|        Guarani|
|  Internacional|
|       Ipatinga|
|      Joinville|
|      Juventude|
|        Nautico|
|      Palmeiras|
|         Parana|
|       Paysandu|
|    Ponte Preta|
|     Portuguesa|
|     Santa Cruz|
|    Santo Andre|
|         Santos|
|    Sao Caetano|
|      Sao Paulo|
|          Sport|
|          Vasco|
|        Vitoria|
+---------------+



In [ ]:
df_jogos.select(F.col('visitante')).distinct().count()

45

In [ ]:
df_jogos.select(F.col('visitante')).distinct().orderBy(F.col('visitante')).show(100)

+---------------+
|      visitante|
+---------------+
|     America-MG|
|     America-RN|
|   Athletico-PR|
|    Atletico-GO|
|    Atletico-MG|
|           Avai|
|          Bahia|
|        Barueri|
|    Botafogo-RJ|
|     Bragantino|
|    Brasiliense|
|            CSA|
|          Ceara|
|    Chapecoense|
|    Corinthians|
|       Coritiba|
|       Criciuma|
|       Cruzeiro|
|         Cuiaba|
|    Figueirense|
|       Flamengo|
|     Fluminense|
|      Fortaleza|
|          Goias|
|         Gremio|
|Gremio Prudente|
|        Guarani|
|  Internacional|
|       Ipatinga|
|      Joinville|
|      Juventude|
|        Nautico|
|      Palmeiras|
|         Parana|
|       Paysandu|
|    Ponte Preta|
|     Portuguesa|
|     Santa Cruz|
|    Santo Andre|
|         Santos|
|    Sao Caetano|
|      Sao Paulo|
|          Sport|
|          Vasco|
|        Vitoria|
+---------------+



In [ ]:
df_jogos.select(F.col('formacao_mandante')).distinct().orderBy(F.col('formacao_mandante')).show(100)

+-----------------+
|formacao_mandante|
+-----------------+
|             null|
|          3-1-4-2|
|          3-2-4-1|
|          3-4-1-2|
|          3-4-2-1|
|            3-4-3|
|          3-5-1-1|
|            3-5-2|
|        4-1-2-1-2|
|          4-1-3-2|
|          4-1-4-1|
|          4-2-2-2|
|          4-2-3-1|
|          4-2-4-0|
|          4-3-1-2|
|          4-3-2-1|
|            4-3-3|
|          4-4-1-1|
|            4-4-2|
|            4-5-1|
|            5-3-2|
|            5-4-1|
+-----------------+



In [ ]:
df_jogos.select(F.col('vencedor')).distinct().orderBy(F.col('vencedor')).show(100)

+---------------+
|       vencedor|
+---------------+
|              -|
|     America-MG|
|     America-RN|
|   Athletico-PR|
|    Atletico-GO|
|    Atletico-MG|
|           Avai|
|          Bahia|
|        Barueri|
|    Botafogo-RJ|
|     Bragantino|
|    Brasiliense|
|            CSA|
|          Ceara|
|    Chapecoense|
|    Corinthians|
|       Coritiba|
|       Criciuma|
|       Cruzeiro|
|         Cuiaba|
|    Figueirense|
|       Flamengo|
|     Fluminense|
|      Fortaleza|
|          Goias|
|         Gremio|
|Gremio Prudente|
|        Guarani|
|  Internacional|
|       Ipatinga|
|      Joinville|
|      Juventude|
|        Nautico|
|      Palmeiras|
|         Parana|
|       Paysandu|
|    Ponte Preta|
|     Portuguesa|
|     Santa Cruz|
|    Santo Andre|
|         Santos|
|    Sao Caetano|
|      Sao Paulo|
|          Sport|
|          Vasco|
|        Vitoria|
+---------------+



In [ ]:
df_jogos.select(F.col('mandante_Estado')).distinct().orderBy(F.col('mandante_Estado')).show(200, truncate = False)

+---------------+
|mandante_Estado|
+---------------+
|AL             |
|BA             |
|CE             |
|DF             |
|GO             |
|MG             |
|MT             |
|PA             |
|PE             |
|PR             |
|RJ             |
|RN             |
|RS             |
|SC             |
|SP             |
+---------------+



In [ ]:
df_jogos.select(F.col('visitante_Estado')).distinct().orderBy(F.col('visitante_Estado')).show(200, truncate = False)

+----------------+
|visitante_Estado|
+----------------+
|AL              |
|BA              |
|CE              |
|DF              |
|GO              |
|MG              |
|MT              |
|PA              |
|PE              |
|PR              |
|RJ              |
|RN              |
|RS              |
|SC              |
|SP              |
+----------------+



In [ ]:
df_jogos.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- rodata: integer (nullable = true)
 |-- data: string (nullable = true)
 |-- hora: timestamp (nullable = true)
 |-- mandante: string (nullable = true)
 |-- visitante: string (nullable = true)
 |-- formacao_mandante: string (nullable = true)
 |-- formacao_visitante: string (nullable = true)
 |-- tecnico_mandante: string (nullable = true)
 |-- tecnico_visitante: string (nullable = true)
 |-- vencedor: string (nullable = true)
 |-- arena: string (nullable = true)
 |-- mandante_Placar: integer (nullable = true)
 |-- visitante_Placar: integer (nullable = true)
 |-- mandante_Estado: string (nullable = true)
 |-- visitante_Estado: string (nullable = true)



In [ ]:
df_backup = df_jogos

In [ ]:
df_jogos = df_backup

In [ ]:
df_jogos = df_jogos.drop('arena', 'hora', 'formacao_mandante', 'formacao_visitante', 'tecnico_mandante', 'tecnico_visitante')

In [ ]:
df_jogos = df_jogos.withColumn('data', F.date_format(F.to_date('data', 'd/M/yyyy'), 'dd/MM/yyyy'))

In [ ]:
df_jogos.show()

+---+------+----------+-------------+-------------+-------------+---------------+----------------+---------------+----------------+
| ID|rodata|      data|     mandante|    visitante|     vencedor|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|
+---+------+----------+-------------+-------------+-------------+---------------+----------------+---------------+----------------+
|  1|     1|29/03/2003|      Guarani|        Vasco|      Guarani|              4|               2|             SP|              RJ|
|  2|     1|29/03/2003| Athletico-PR|       Gremio| Athletico-PR|              2|               0|             PR|              RS|
|  3|     1|30/03/2003|     Flamengo|     Coritiba|            -|              1|               1|             RJ|              PR|
|  4|     1|30/03/2003|        Goias|     Paysandu|            -|              2|               2|             GO|              PA|
|  5|     1|30/03/2003|Internacional|  Ponte Preta|            -|           

In [ ]:
df_jogos = df_jogos.withColumn("data", F.to_date("data", "dd/MM/yyyy"))
df_jogos = df_jogos.withColumn("data", df_jogos["data"].cast(DateType()))

In [ ]:
df_jogos.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- rodata: integer (nullable = true)
 |-- data: date (nullable = true)
 |-- mandante: string (nullable = true)
 |-- visitante: string (nullable = true)
 |-- vencedor: string (nullable = true)
 |-- mandante_Placar: integer (nullable = true)
 |-- visitante_Placar: integer (nullable = true)
 |-- mandante_Estado: string (nullable = true)
 |-- visitante_Estado: string (nullable = true)



Quantidade de clubes que participavam do Brasileirão por ano:

|2003| - 24\
|2004| - 24\
|2005| - 22\
|2006| - 20\
|2007| - 20\
|2008| - 20\
|2009| - 20\
|2010| - 20\
|2011| - 20\
|2012| - 20\
|2013| - 20\
|2014| - 20\
|2015| - 20\
|2016| - 20\
|2017| - 20\
|2018| - 20\
|2019| - 20\
|2020| - 20\
|2021| - 20\
|2022| - 20

Portanto irei desconsiderar os anos de 2003,2004 e 2005 por terem um formato diferente do atual.

In [ ]:
df_jogos = df_jogos.filter((F.year('data') != 2003) & (F.year('data') != 2004) & (F.year('data') != 2005))

In [ ]:
nulos = []
for coluna in df_jogos.columns:
    null_count = df_jogos.where(F.col(coluna).isNull()).count()
    nulos.append((coluna, null_count))

# Exibir a quantidade de valores nulos em cada coluna
for coluna, null_count in nulos:
    print(f"A coluna {coluna} possui {null_count} valores nulos.")

A coluna ID possui 0 valores nulos.
A coluna rodata possui 0 valores nulos.
A coluna data possui 0 valores nulos.
A coluna mandante possui 0 valores nulos.
A coluna visitante possui 0 valores nulos.
A coluna vencedor possui 0 valores nulos.
A coluna mandante_Placar possui 0 valores nulos.
A coluna visitante_Placar possui 0 valores nulos.
A coluna mandante_Estado possui 0 valores nulos.
A coluna visitante_Estado possui 0 valores nulos.


In [ ]:
df_jogos.select(F.year(F.col('data')).alias('ano')).groupBy('ano').count().orderBy('ano').show()

+----+-----+
| ano|count|
+----+-----+
|2006|  380|
|2007|  380|
|2008|  380|
|2009|  380|
|2010|  380|
|2011|  380|
|2012|  380|
|2013|  380|
|2014|  380|
|2015|  380|
|2016|  379|
|2017|  380|
|2018|  380|
|2019|  380|
|2020|  268|
|2021|  492|
|2022|  380|
+----+-----+



In [ ]:
df_jogos.show()

+----+------+----------+-------------+-------------+-------------+---------------+----------------+---------------+----------------+
|  ID|rodata|      data|     mandante|    visitante|     vencedor|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|
+----+------+----------+-------------+-------------+-------------+---------------+----------------+---------------+----------------+
|1567|     1|2006-04-15|    Juventude|       Parana|    Juventude|              1|               0|             RS|              PR|
|1568|     1|2006-04-15|        Vasco|Internacional|            -|              1|               1|             RJ|              RS|
|1569|     1|2006-04-16|    Sao Paulo|     Flamengo|    Sao Paulo|              1|               0|             SP|              RJ|
|1570|     1|2006-04-16|       Gremio|  Corinthians|       Gremio|              2|               0|             RS|              SP|
|1571|     1|2006-04-16|  Botafogo-RJ|    Fortaleza|  Botafogo-RJ|   

In [ ]:
df_jogos.select(F.col('vencedor')).groupBy('vencedor').count().orderBy(F.desc('count')).show()

+-------------+-----+
|     vencedor|count|
+-------------+-----+
|            -| 1738|
|     Flamengo|  290|
|    Sao Paulo|  289|
|       Gremio|  273|
|    Palmeiras|  267|
|  Atletico-MG|  264|
|       Santos|  261|
|Internacional|  261|
|   Fluminense|  256|
|  Corinthians|  255|
| Athletico-PR|  235|
|     Cruzeiro|  229|
|  Botafogo-RJ|  201|
|        Vasco|  155|
|        Goias|  139|
|     Coritiba|  132|
|        Sport|  131|
|        Bahia|  103|
|      Vitoria|   95|
|  Figueirense|   94|
+-------------+-----+
only showing top 20 rows



In [ ]:
#Primeira rodada do ano de 2015 que é onde começam a ter dados estatísticos
df_jogos.filter(df_jogos.ID == 4987).show()

+----+------+----------+-----------+---------+-----------+---------------+----------------+---------------+----------------+
|  ID|rodata|      data|   mandante|visitante|   vencedor|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|
+----+------+----------+-----------+---------+-----------+---------------+----------------+---------------+----------------+
|4987|     1|2015-05-09|Chapecoense| Coritiba|Chapecoense|              2|               1|             SC|              PR|
+----+------+----------+-----------+---------+-----------+---------------+----------------+---------------+----------------+



Analise do df_estatisticas

Só existem dados estatísticos das partidas a partir de 2015

In [ ]:
df_estatisticas = df_estatisticas.filter(df_estatisticas.partida_id >= 4987)
df_estatisticas = df_estatisticas.drop('precisao_passes', 'chutes_no_alvo')

In [ ]:
df_estatisticas.show(truncate = False)

+----------+------+-------------+------+-------------+------+------+--------------+---------------+------------+----------+
|partida_id|rodata|clube        |chutes|posse_de_bola|passes|faltas|cartao_amarelo|cartao_vermelho|impedimentos|escanteios|
+----------+------+-------------+------+-------------+------+------+--------------+---------------+------------+----------+
|4987      |1     |Chapecoense  |13    |45%          |326   |20    |2             |0              |3           |3         |
|4987      |1     |Coritiba     |16    |55%          |388   |21    |2             |0              |0           |5         |
|4988      |1     |Palmeiras    |17    |50%          |406   |20    |2             |0              |2           |11        |
|4988      |1     |Atletico-MG  |10    |50%          |426   |14    |4             |0              |4           |2         |
|4989      |1     |Fluminense   |26    |74%          |702   |9     |2             |0              |1           |5         |
|4989   

In [ ]:
nulos = []
for coluna in df_estatisticas.columns:
    null_count = df_estatisticas.where(F.col(coluna).isNull()).count()
    nulos.append((coluna, null_count))

# Exibir a quantidade de valores nulos em cada coluna
for coluna, null_count in nulos:
    print(f"A coluna {coluna} possui {null_count} valores nulos.")

A coluna partida_id possui 0 valores nulos.
A coluna rodata possui 0 valores nulos.
A coluna clube possui 0 valores nulos.
A coluna chutes possui 0 valores nulos.
A coluna posse_de_bola possui 0 valores nulos.
A coluna passes possui 0 valores nulos.
A coluna faltas possui 0 valores nulos.
A coluna cartao_amarelo possui 0 valores nulos.
A coluna cartao_vermelho possui 0 valores nulos.
A coluna impedimentos possui 0 valores nulos.
A coluna escanteios possui 0 valores nulos.


In [ ]:
df_estatisticas.printSchema()

root
 |-- partida_id: integer (nullable = true)
 |-- rodata: integer (nullable = true)
 |-- clube: string (nullable = true)
 |-- chutes: integer (nullable = true)
 |-- posse_de_bola: string (nullable = true)
 |-- passes: integer (nullable = true)
 |-- faltas: integer (nullable = true)
 |-- cartao_amarelo: integer (nullable = true)
 |-- cartao_vermelho: integer (nullable = true)
 |-- impedimentos: integer (nullable = true)
 |-- escanteios: integer (nullable = true)



## Join dos jogos com as estatisticas

In [ ]:
df_completo = df_jogos.join(df_estatisticas, df_jogos.ID == df_estatisticas.partida_id, 'inner')

In [ ]:
df_completo.show()

+----+------+----------+------------+-------------+------------+---------------+----------------+---------------+----------------+----------+------+-------------+------+-------------+------+------+--------------+---------------+------------+----------+
|  ID|rodata|      data|    mandante|    visitante|    vencedor|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|partida_id|rodata|        clube|chutes|posse_de_bola|passes|faltas|cartao_amarelo|cartao_vermelho|impedimentos|escanteios|
+----+------+----------+------------+-------------+------------+---------------+----------------+---------------+----------------+----------+------+-------------+------+-------------+------+------+--------------+---------------+------------+----------+
|4987|     1|2015-05-09| Chapecoense|     Coritiba| Chapecoense|              2|               1|             SC|              PR|      4987|     1|  Chapecoense|    13|          45%|   326|    20|             2|              0|           3|

In [ ]:
df_completo.count()

6078

In [ ]:
df_completo.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- rodata: integer (nullable = true)
 |-- data: date (nullable = true)
 |-- mandante: string (nullable = true)
 |-- visitante: string (nullable = true)
 |-- vencedor: string (nullable = true)
 |-- mandante_Placar: integer (nullable = true)
 |-- visitante_Placar: integer (nullable = true)
 |-- mandante_Estado: string (nullable = true)
 |-- visitante_Estado: string (nullable = true)
 |-- partida_id: integer (nullable = true)
 |-- rodata: integer (nullable = true)
 |-- clube: string (nullable = true)
 |-- chutes: integer (nullable = true)
 |-- posse_de_bola: string (nullable = true)
 |-- passes: integer (nullable = true)
 |-- faltas: integer (nullable = true)
 |-- cartao_amarelo: integer (nullable = true)
 |-- cartao_vermelho: integer (nullable = true)
 |-- impedimentos: integer (nullable = true)
 |-- escanteios: integer (nullable = true)



##Separando apenas o dataframe para fazer análises direcionadas apenas ao Flamengo

In [ ]:
df_fla = df_jogos.filter((df_jogos.mandante == 'Flamengo') | (df_jogos.visitante == 'Flamengo'))

In [ ]:
df_fla.show(38)

+----+------+----------+-------------+-------------+-------------+---------------+----------------+---------------+----------------+
|  ID|rodata|      data|     mandante|    visitante|     vencedor|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|
+----+------+----------+-------------+-------------+-------------+---------------+----------------+---------------+----------------+
|1569|     1|2006-04-16|    Sao Paulo|     Flamengo|    Sao Paulo|              1|               0|             SP|              RJ|
|1583|     2|2006-04-23|     Flamengo|    Juventude|     Flamengo|              3|               1|             RJ|              RS|
|1591|     3|2006-04-30|Internacional|     Flamengo|Internacional|              1|               0|             RS|              RJ|
|1604|     4|2006-05-07|     Flamengo|  Botafogo-RJ|     Flamengo|              1|               0|             RJ|              RJ|
|1614|     5|2006-05-14|     Flamengo|    Fortaleza|            -|   

In [ ]:
df_fla = df_fla.withColumn('vencedor',
                           F.when(df_fla['vencedor'] == 'Flamengo', 'Vitória')
                           .when(df_fla['vencedor'] == '-', 'Empate')
                           .otherwise('Derrota'))

In [ ]:
df_fla = df_fla.withColumnRenamed('vencedor', 'situacao_flamengo')

In [ ]:
df_fla.show()

+----+------+----------+-------------+-----------+-----------------+---------------+----------------+---------------+----------------+
|  ID|rodata|      data|     mandante|  visitante|situacao_flamengo|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|
+----+------+----------+-------------+-----------+-----------------+---------------+----------------+---------------+----------------+
|1569|     1|2006-04-16|    Sao Paulo|   Flamengo|          Derrota|              1|               0|             SP|              RJ|
|1583|     2|2006-04-23|     Flamengo|  Juventude|          Vitória|              3|               1|             RJ|              RS|
|1591|     3|2006-04-30|Internacional|   Flamengo|          Derrota|              1|               0|             RS|              RJ|
|1604|     4|2006-05-07|     Flamengo|Botafogo-RJ|          Vitória|              1|               0|             RJ|              RJ|
|1614|     5|2006-05-14|     Flamengo|  Fortaleza|     

In [ ]:
df_fla_est = df_estatisticas.filter(df_estatisticas.clube == 'Flamengo')

In [ ]:
df_fla_est.show()

+----------+------+--------+------+-------------+------+------+--------------+---------------+------------+----------+
|partida_id|rodata|   clube|chutes|posse_de_bola|passes|faltas|cartao_amarelo|cartao_vermelho|impedimentos|escanteios|
+----------+------+--------+------+-------------+------+------+--------------+---------------+------------+----------+
|      4992|     1|Flamengo|    12|          46%|   430|    12|             1|              0|           6|         6|
|      5002|     2|Flamengo|    18|          59%|   492|    10|             2|              0|           6|        13|
|      5014|     3|Flamengo|     8|          57%|   443|    17|             6|              0|           7|         2|
|      5025|     4|Flamengo|    15|          67%|   475|    24|             5|              1|           1|         8|
|      5031|     5|Flamengo|     8|          48%|   378|    17|             3|              0|           1|        10|
|      5038|     6|Flamengo|    18|          64%

In [ ]:
df_completo_fla = df_fla.join(df_fla_est, df_fla.ID == df_fla_est.partida_id, 'inner')

In [ ]:
df_completo_fla = df_completo_fla.drop('ID')

In [ ]:
df_completo_fla.show()

+------+----------+-------------+------------+-----------------+---------------+----------------+---------------+----------------+----------+------+--------+------+-------------+------+------+--------------+---------------+------------+----------+
|rodata|      data|     mandante|   visitante|situacao_flamengo|mandante_Placar|visitante_Placar|mandante_Estado|visitante_Estado|partida_id|rodata|   clube|chutes|posse_de_bola|passes|faltas|cartao_amarelo|cartao_vermelho|impedimentos|escanteios|
+------+----------+-------------+------------+-----------------+---------------+----------------+---------------+----------------+----------+------+--------+------+-------------+------+------+--------------+---------------+------------+----------+
|     1|2015-05-10|    Sao Paulo|    Flamengo|          Derrota|              2|               1|             SP|              RJ|      4992|     1|Flamengo|    12|          46%|   430|    12|             1|              0|           6|         6|
|     2|

##Load (Subindo df tratado para o Drive)



In [ ]:
df_salvar = df_completo.toPandas()

In [ ]:
gcs_path = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_brasileirao_com_estatistica_seppevtratado.csv'
gcs_path2 = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_brasileirao_com_estatistica_sepvirg_tratado.csv'
df_salvar.to_csv(gcs_path, sep = ';', index = False)
df_salvar.to_csv(gcs_path2, sep = ',', index = False)

In [ ]:
df_salvar_jogos = df_jogos.toPandas()
df_salvar_est = df_estatisticas.toPandas()

In [ ]:
gcs_path_jogos = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_jogos_brasileirao_seppevtratado.csv'
gcs_path_jogos2 = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_jogos_brasileirao_sepvirg_tratado.csv'
df_salvar_jogos.to_csv(gcs_path_jogos, sep = ';', index = False)
df_salvar_jogos.to_csv(gcs_path_jogos2, sep = ',', index = False)

In [ ]:
gcs_path_est = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_estatisticas_brasileirao_seppevtratado.csv'
gcs_path_est2 = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_estatisticas_brasileirao_sepvirg_tratado.csv'
df_salvar_est.to_csv(gcs_path_est, sep = ';', index = False)
df_salvar_est.to_csv(gcs_path_est2, sep = ',', index = False)

##Load dfs só com jogos do Flamengo

In [ ]:
df_fla_pd = df_fla.toPandas()
df_fla_est_pd = df_fla_est.toPandas()
df_completo_fla_pd = df_completo_fla.toPandas()

In [ ]:
drive_path = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_fla_brasileirao_tratado.csv'
drive_path2 = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_fla_brasileirao_estatistica_tratado.csv'
drive_path3 = '/content/drive/MyDrive/Projeto Braileirao/tratados/dados_fla_brasileirao_completo_tratado.csv'
df_fla_pd.to_csv(drive_path, sep = ';', index = False)
df_fla_est_pd.to_csv(drive_path2, sep = ';', index = False)
df_completo_fla_pd.to_csv(drive_path3, sep = ';', index = False)

Link para visualização do Dashboard feito com as informações do Clube Flamengo:

https://app.powerbi.com/view?r=eyJrIjoiNDhlMzA5YWMtZDZjYy00ZjAxLWE0NmMtMzFiMzBiOTk0MTljIiwidCI6IjM1ZWFlNjdjLTY4NDctNDA5Mi1hMDk5LThkMGVhYTM1Nzc3ZiJ9&pageName=ReportSection